In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
from datetime import datetime
import matplotlib.pyplot as plt
import dataset_fctns, plotting

In [2]:
def phase_dependent_response(driver_values, t_dev, responses, thresholds):
    #Thresholds are the thresholds in development time where the different growth phases change
    #Responses are the response functions, index starting at 'before the first threshold'
    #driver values are the inputs to the response function
    #t_dev is the (cts) development time
    phase = np.digitize(t_dev, thresholds)
    response = np.zeros(driver_values.shape)
    for phase_index in range(len(responses)):
        response += (phase == phase_index)*responses[phase_index](driver_values) #First brackets indicates if we are at the right phase, second takes the response function for each phase
    return response

In [3]:
def get_nearest_elements(x, y, ds):
    #Interpolates the input array onto the (non-gridded e.g. phenology station) coordinates x and y.
    #Note hyras is not stored on the full grid, but on some kind of subset. Not quite sure how this works. Just got to hope the stations are in a hyras gridpoint.
    X_for_interp = xr.DataArray(x, dims="modelpoint")
    Y_for_interp = xr.DataArray(y, dims="modelpoint")
    return ds.interp(x=X_for_interp, y=Y_for_interp)#, kwargs={"fill_value": None})

In [4]:
def latlon_to_projection(x_coords, y_coords):
    proj_epsg = ccrs.epsg(3034)
    proj_latlon = ccrs.PlateCarree()
    points_epsg = proj_epsg.transform_points(proj_latlon, x_coords, y_coords)
    x_epsg = points_epsg[:, 0]
    y_epsg = points_epsg[:, 1]
    return x_epsg, y_epsg

In [5]:
def make_input_array(emergence_times, station_ids, x_coords, y_coords, ds_driver):
    #Put in transforming to the right coords here x
    print('project to new coords')
    x_epsg, y_epsg = latlon_to_projection(x_coords, y_coords)
    print('interpolate driver to station locations')
    ds_driver_interp = get_nearest_elements(x_epsg, y_epsg, ds_driver)
    ds_driver_interp['emergence_dates'] = (("modelpoint"), emergence_times)
    ds_driver_interp['Stations_id'] = (("modelpoint"), station_ids)
    return ds_driver_interp#.dropna(dim='modelpoint')


In [6]:
def dev_under_response(response, ds_driver, thresholds, maturity_t_dev, emergence_date):
    # Response is the rate response to driver values. Driver values are the input to this response. Maturity_t_dev is the t_dev value where we should stop running.
    # Put in the date as (day, month, year)
    t_dev = np.zeros(ds_driver.isel(time=0).values.shape) #Continuous development time. When this passes through some thresholds then have change in phase.
    dev_time_series = [t_dev.copy()]
    #i_day = np.datetime64(f'{emergence_date[2]}-{str(emergence_date[1]).zfill(2)}-{str(emergence_date[0]).zfill(2)}')
    i_day = ds_driver['emergence_dates']
    while np.any(t_dev < maturity_t_dev):
        i_day += np.timedelta64(1,'D')
        #driver_values = ds_driver.sel(time=np.datetime_as_string(i_day, unit='D')).squeeze().values
        driver_values = ds_driver.sel(time=i_day).squeeze().values
        t_dev += response(driver_values, t_dev) #Rate of change of development stage
        dev_time_series.append(t_dev.copy())
    return np.stack(dev_time_series)

In [7]:
def get_phase_dates(dev_time_series, thresholds):
    phase_dates_array = np.zeros((len(thresholds), dev_time_series.shape[1], dev_time_series.shape[2]))
    for i_x in range(dev_time_series.shape[1]):
        for i_y in range(dev_time_series.shape[2]):
            phase_dates_array[:, i_x, i_y] = np.digitize(thresholds, dev_time_series[:, i_x, i_y]) #Note that the thresholds are NOT the bins for numpy digitize!
    return phase_dates_array

In [15]:
## READ IN DATA ##
ds_mean = xr.open_dataset('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\tas_hyras_5_1951_2020_v5-0_de.nc')
phen_data_maize = dataset_fctns.read_phen_dataset("C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\PH_Jahresmelder_Landwirtschaft_Kulturpflanze_Mais_1936_2023_hist.txt", drop_list = ['Unnamed: 9'])
phase_names = pd.read_csv("https://opendata.dwd.de/climate_environment/CDC/help/PH_Beschreibung_Phase.txt", encoding = "latin1", engine='python', sep = r';\s+|;\t+|;\s+\t+|;\t+\s+|;|\s+;|\t+;|\s+\t+;|\t+\s+;')
station_data = pd.read_csv("https://opendata.dwd.de/climate_environment/CDC/help/PH_Beschreibung_Phaenologie_Stationen_Jahresmelder.txt",sep = ";\s+|;\t+|;\s+\t+|;\t+\s+|;|\s+;|\t+;|\s+\t+;|\t+\s+;", encoding='cp1252', on_bad_lines='skip')
station_data = station_data.drop('Unnamed: 12', axis = 1)
## READ IN DATA FOR PHASE NAMES IN ORDER ##
#print(phase_names['Phase_ID'])
phen_data_maize = dataset_fctns.add_locations(phen_data_maize, station_data)
phen_data_maize = dataset_fctns.phase_order_name(phen_data_maize, phase_names, [10, 12, 67, 65, 5, 6, 19, 20, 21, 24, ])
phen_data_maize = dataset_fctns.order_phen_dataset(phen_data_maize)
#phen_data_wheat = dataset_fctns.add_locations(phen_data_wheat, station_data)
#phen_data_wheat = dataset_fctns.phase_order_name(phen_data_wheat, phase_names, [10, 12, 15, 18, 19, 21, 22, 23, 24, ])
#phen_data_wheat = dataset_fctns.order_phen_dataset(phen_data_wheat)
just_emergence_phen_data = phen_data_maize.where(phen_data_maize['Name of phase'] == 'beginning of emergence').dropna()
just_emergence_phen_data = just_emergence_phen_data.where(just_emergence_phen_data['Eintrittsdatum'] > np.datetime64('2005-01-01')).dropna()
emergences = just_emergence_phen_data['Eintrittsdatum'].values
station_ids = just_emergence_phen_data['Stations_id'].values
x_coords = just_emergence_phen_data['lon'].values
y_coords = just_emergence_phen_data['lat'].values

C:\Users\wlwc1989\AppData\Local\Temp\ipykernel_22904\227109524.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  station_data = pd.read_csv("https://opendata.dwd.de/climate_environment/CDC/help/PH_Beschreibung_Phaenologie_Stationen_Jahresmelder.txt",sep = ";\s+|;\t+|;\s+\t+|;\t+\s+|;|\s+;|\t+;|\s+\t+;|\t+\s+;", encoding='cp1252', on_bad_lines='skip')


In [16]:
ds_driver = make_input_array(emergences, station_ids, x_coords, y_coords, ds_mean)

project to new coords
interpolate driver to station locations


Need:
 - Get emergence times from DWD dataset
 - Get realistic temperature response functions and put these into function

In [205]:
x = np.array([10.5, 12])
y = np.array([49, 51])
emergence_times = np.array([np.datetime64('2000-04-15'), np.datetime64('2000-05-19')])
ds_input = make_input_array(emergence_times, x, y, ds_mean)

C:\Users\wlwc1989\AppData\Local\Temp\ipykernel_20560\3568550490.py:5: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  ds_driver_interp['emergence_dates'] = (("modelpoint"), emergence_times)
C:\Users\wlwc1989\AppData\Local\Temp\ipykernel_20560\3568550490.py:5: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to 

In [204]:
ds_input

<xarray.Dataset>
Dimensions:             (time: 25568, modelpoint: 2, bnds: 2)
Coordinates:
  * time                (time) datetime64[ns] 1951-01-01T12:00:00 ... 2020-12...
    lon                 (modelpoint) float64 10.0 12.0
    lat                 (modelpoint) float64 49.0 51.0
    x                   (modelpoint) float64 4e+06 4.136e+06
    y                   (modelpoint) float64 2.478e+06 2.694e+06
Dimensions without coordinates: modelpoint, bnds
Data variables:
    time_bnds           (time, bnds) datetime64[ns] 1951-01-01 ... 2021-01-01
    x_bnds              (modelpoint, bnds) float64 3.998e+06 ... 4.138e+06
    y_bnds              (modelpoint, bnds) float64 2.475e+06 ... 2.697e+06
    crs_HYRAS           int32 1
    tas                 (time, modelpoint) float64 0.4 -0.601 ... 0.2 1.723
    number_of_stations  (time) float64 817.0 818.0 819.0 ... 1.454e+03 1.456e+03
    emergence_dates     (modelpoint) datetime64[ns] 2000-04-15 2000-05-19
Attributes: (12/19)
    source:                 surface observations
    Conventions:            CF-1.8
    realization:            v5.0
    project_id:             HYRAS
    input_data_status:      non-realtime
    level_type:             surface
    ...                     ...
    horizontal_resolution:  5 km
    title:                  gridded_temperature_dataset_(HYRAS-DE TAS)
    institution:            Deutscher Wetterdienst (DWD)
    creation_date:          created at 2022-12-04 13:12:56
    filename:               tas_hyras_5_1951_2020_v5-0_de.nc
    unique_dataset_id:      DWD_HYRAS_DE_tas_v5.0_1951_2020_day_00638C9CD8

Need:
 - More realistic temperature response functions
 - What to do about emergence date
 - Relate to phen data?

In [ ]:
ds_mean = xr.open_dataset('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\Saved_files\\tas_hyras_5_1951_2020_v5-0_de.nc')
#ds_mean = ds_mean.where((ds_mean['x'] > 4020000)*(ds_mean['x'] < 4080000)*(ds_mean['y'] > 2520000)*(ds_mean['y'] > 2580000), drop = True)
resps = [lambda x: 2*x*(x > 0) + 1*(x<= 0), lambda x: 0.5*x*(x > 0) + 1*(x<= 0), lambda x: 0.7*x*(x > 0) + 1*(x<= 0), lambda x: 0*x]
Ts = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
thresholds = [3, 6, 10]
ts = dev_under_response(lambda x, y: phase_dependent_response(x, y, resps, thresholds), ds_mean['tas'], [3, 6, 10], 10, (15, 4, 2000)) #ds_mean['tas']
pds = get_phase_dates(ts, [3, 6, 10])

In [ ]:
dev_under_response(lambda x, y: phase_dependent_response(x, y, resps, thresholds), ds_mean['tas'], [3, 6, 10], 10, 15000)#np.datetime64('2000-03-25')
#[lambda x: 2*x, lambda x: 0.5*x, lambda x: 0.1*x, lambda x: 0*x]

In [41]:
resps = [lambda x: 2*x*(x > 0) + 1*(x<= 0), lambda x: 0.5*x*(x > 0) + 1*(x<= 0), lambda x: 0.7*x*(x > 0) + 1*(x<= 0), lambda x: 0*x]
Ts = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
thresholds = [3, 6, 10]
#print(phase_dependent_response(Ts, Ts, resps,  thresholds))